# Test the ENC-DEC model

In [815]:
import dataloader
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import models
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
import os
import time
from torch_geometric.nn import GATv2Conv, global_mean_pool
# reload library
import importlib
import cv2
import utils as ut
import pandas as pd
import DataDLC

In [816]:
importlib.reload(dataloader)
importlib.reload(DataDLC)
importlib.reload(models)

<module 'models' from 'c:\\Users\\jalvarez\\Documents\\Code\\GitHubCOde\\Behavioral_Tagging_of_Mice_in_multiple_Mice_dataset_using_Deep_Learning\\src\\models.py'>

## Load dataset

**Obs:** This is just a test to see if the model is working.

In [817]:
importlib.reload(dataloader)
importlib.reload(DataDLC)

<module 'DataDLC' from 'c:\\Users\\jalvarez\\Documents\\Code\\GitHubCOde\\Behavioral_Tagging_of_Mice_in_multiple_Mice_dataset_using_Deep_Learning\\src\\DataDLC.py'>

In [632]:

dataloader.reload_module()

In [633]:
# Load dataset .pt files
import pickle as pkl


In [665]:
# deactivate warnings

import warnings
warnings.filterwarnings("ignore")


data_loader = dataloader.DLCDataLoader(r'c:\Users\jalvarez\Documents\Data\DataLoadaerTESTTSTST', load_dataset=False, batch_size=1, num_workers=0, device='cpu', window_size=1, stride = 1, build_graph=True, behaviour='General_Contacts')


['DMD_mal_Test_1DLC_dlcrnetms5_More_BodyPartsJul9shuffle1_740000_el_filtered.h5']
Loading data from c:\Users\jalvarez\Documents\Data\DataLoadaerTESTTSTST, where we have 1 files
We have 1 files
Loading file DMD_mal_Test_1DLC_dlcrnetms5_More_BodyPartsJul9shuffle1_740000_el_filtered.h5


100%|██████████| 2632/2632 [00:32<00:00, 80.68it/s]

Number of files: 1


In [666]:
DLoader = data_loader.get_dataloader()

In [ ]:
# Split train and test
train_size = int(0.8 * len(DLoader))
test_size = len(DLoader) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(DLoader, [train_size, test_size])


In [667]:
DLoader.batch_size

1

In [668]:
for data in DLoader:
    a = data
    break

In [669]:
a.behaviour

tensor([0])

---

# Create Graphs

This will take a while, and i dont even know if it is going to work. jejeje

In [865]:
importlib.reload(models)

<module 'models' from 'c:\\Users\\jalvarez\\Documents\\Code\\GitHubCOde\\Behavioral_Tagging_of_Mice_in_multiple_Mice_dataset_using_Deep_Learning\\src\\models.py'>

In [866]:
len(dataset)

2628

In [867]:
def concatenate_per_graph(embbed, batch):
    ''' Concatenate the embeddings per graph '''
    out = []
    for i in range(batch.max()+1):
        out.append(embbed[batch==i].flatten())
    return out

In [868]:
for data in DLoader:
    a = data
    break

### Graph Classifier

In [869]:
graphencoder = models.GATEncoder(nout = 32, nhid=16, attention_hidden=2, n_in=4, dropout=0.5)
class_head = models.ClassificationHead(n_latent=576, nhid = 64, nout = 2)

In [870]:
lat = graphencoder(a.x, a.edge_index, a.frame_mask)

In [871]:
lat.shape

torch.Size([36, 16])

In [872]:
# Flatten the latent representation keeping the batch dimension
#lat = global_mean_pool(lat, a.batch, size = a.batch.max().item()+1)
lat.flatten().shape


torch.Size([576])

In [873]:
conc_embbed = concatenate_per_graph(lat, a.batch)

In [874]:
len(conc_embbed)

1

In [875]:
conc_embbed[0].shape

torch.Size([576])

In [876]:
lat.shape

torch.Size([36, 16])

In [877]:
out = class_head(conc_embbed[0])

In [878]:
out.shape

torch.Size([2])

In [879]:
model = models.GraphClassifier(graphencoder, class_head)

In [880]:
out = model(a)
out.dtype
a = torch.tensor(a.behaviour, dtype=torch.long)

In [881]:
ou

NameError: name 'ou' is not defined

In [882]:
a.x.flatten().shape

AttributeError: 'Tensor' object has no attribute 'x'

In [859]:
out.shape

torch.Size([2])

In [883]:
print('The model has %d trainable parameters' % sum(p.numel() for p in model.parameters() if p.requires_grad))

The model has 40658 trainable parameters


In [884]:
import tqdm

#### Trainning Loop

In [885]:
# Trainning loop
epochs = 50
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

model.train()
for epoch in range(epochs):
    print('Epoch %d' % epoch)
    running_loss = 0.0
    for i, data in tqdm.tqdm(enumerate(DLoader, 0)):
        optimizer.zero_grad()
        out = model(data).unsqueeze(0)
        loss = criterion(out, data.behaviour)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch %d, loss: %.3f' % (epoch, running_loss))

Epoch 0


2632it [00:31, 82.29it/s]


Epoch 0, loss: 1771.654
Epoch 1


2632it [00:32, 80.84it/s]


Epoch 1, loss: 1662.537
Epoch 2


...

Epoch 15, loss: 1523.693
Epoch 16


2632it [00:32, 81.27it/s]


Epoch 16, loss: 1533.489
Epoch 17


1901it [00:24, 77.81it/s]


KeyboardInterrupt: 

In [745]:
out = GraphAE.forward(data_loader.dataset[0].x, data_loader.dataset[0].edge_index, data_loader.dataset[0].frame_mask)

NameError: name 'GraphAE' is not defined

In [ ]:
# Loss and optimizer
loss = GraphAE.loss(data_loader.dataset[0].x, out)

In [ ]:
print(loss)

# Trainning loop

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
encoder = models.GATEncoder(nout = 64, nhid=16, attention_hidden=2, n_in=3, dropout=0.5).to(device)
print(encoder)
decoder = models.GATDecoder(n_latent=64, n_hidden=16, n_out=3).to(device)
print(decoder)
model = models.GraphAE(encoder, decoder).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
data = data_loader.dataset

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    for i in range(len(data)):

        out = model(data[i].x.to(device), data[i].edge_index.to(device), data[i].frame_mask.to(device))
        loss = model.loss(data[i].x, out)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}, Loss {loss.item()}')

model.eval()
out = model(data[0].x, data[0].edge_index, data[0].frame_mask)

In [ ]:
print(out[0][0].shape)

In [ ]:
numpyout = out[0][0].detach().numpy()

In [ ]:
# Plot the output of the model, the first dimension are the points, and the second one is the x and y coordinates
plt.scatter(numpyout[:,0], numpyout[:,1])


In [ ]:
print(data_loader.dataset[0].x.shape)
print(x[0].shape)
print(x[1].shape)

In [ ]:
print(data_loader.dataset[0].x.shape)
print(x[0].shape)
print(x[1].shape)

In [ ]:
contacts = behaviour.iloc[:,1]

---

---

---

---

---

---

---

#### eDIT NAMES

In [23]:
path = r'c:\Users\jalvarez\Documents\Data\DEEPLABCUT_needs_this_folder_I_dont_like_dlc'

# replace ' ' by '_'
for filename in os.listdir(path):
    os.rename(os.path.join(path, filename), os.path.join(path, filename.replace(' ', '_')))

---

In [48]:
import pandas as pd

In [54]:
data_dlc = DataDLC.DataDLC(r'c:\Users\jalvarez\Documents\Data\DataLoadaerTESTTSTST\DMD_mal_Test_1DLC_dlcrnetms5_More_BodyPartsJul9shuffle1_740000_el_filtered.h5')

data_dlc.drop_tail_bodyparts()

coords = data_dlc.coords.to_numpy()

In [61]:
coords.shape
# Reshape coords as (n_frames, n_individuals, n_bodyparts, 3)
coords = coords.reshape((coords.shape[0], data_dlc.n_individuals, data_dlc.n_body_parts, 3))

dtype('float64')